Eliminamos los documentos con menos de dos palabras
<h3>Preprocesado</h3
<ol>
    <li>Tokenización y lematización</li>
    <li>Eliminar signos de puntuación, tokens numéricos y stopwords</li>
    <li>Eliminar acentos</li>
    <li>Eliminar tokens de 1 carácter</li>
    <li>Aplicar filter_extremes no_below=7 (Al menos aparezca en 7 documentos), no_above=0.7 (no aparezca en más del 70% de los documentos)</li>
</ol>
<h3>Análisis y Unificación de servicios: </h3>
<ol>
    <li>Análisis de servicios existentes con códigos diferentes</li>
    <li>Unificación de servicios para reducir su número</li>
</ol>    

In [1]:
import pandas as pd
import requests
import datetime
import re
import string
import collections
import spacy
import ast
import unicodedata
from gensim import corpora
from gensim.corpora import MmCorpus
from gensim.models import LdaModel
from collections import Counter
import copy
from sklearn.metrics import silhouette_score, adjusted_rand_score, normalized_mutual_info_score, davies_bouldin_score, calinski_harabasz_score
import bcubed
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
df_original = pd.read_csv('qys_original.csv', sep=";")

In [4]:
def count_words(text):
    words = re.findall(r'\b\w+\b',text)
    return len(words)

df_word = df_original.copy()
df_word['texto'] = df_word['title'].astype(str) + ' ' + df_word['description'].astype(str)
df_word['count_word'] = df_word['texto'].apply(count_words)

In [9]:
df_word[['count_word','texto']]

,count_word,texto
0,271,Equipo Flip-Flap Me dirijo a ustedes aprovecha...
1,239,Instalación deportiva para gimnasia deportiva ...
2,86,"Duda Permisos Hola, escribo en nombre de un gr..."
3,75,Noches Jueves viernes y sábado Cada vez es más...
4,120,Falta de instalaciones deportivas Somos una fa...
...,...,...
66452,42,Reposición o reparación de cubo de basura Buen...
66453,36,Baldosas defectuosas Justo en la puerta de la ...
66454,202,Aviso de que reclamación sigue sin respuesta ...
66455,40,"Recogida de basura Buenos dias, muestro mi ma..."


In [10]:
#eliminamos los documentos con menos de dos palabras
df_word=df_word[df_word['count_word'] >= 3]

In [13]:
df_preprocesado = df_word.copy()

In [3]:
import spacy
import unicodedata

nlp = spacy.load("es_core_news_sm")

def preprocesar_textos(texto, generar_bigramas=False):
    # 1. Tokenización y lematización
    doc = nlp(texto.lower())  # Procesa el texto en minúsculas
    # 2. Eliminar signos de puntuación, tokens numéricos y stopwords
    tokens_sin_stopwords = [token.lemma_ for token in doc if not token.is_punct and token.is_alpha and not token.is_stop]

    # 3. Eliminar acentos
    tokens_sin_acentos = []
    for token in tokens_sin_stopwords:
        if 'ñ' in token or 'Ñ' in token:
            tokens_sin_acentos.append(token)
        else:
            normalized_token = unicodedata.normalize('NFKD', token)
            token_sin_acento = ''.join([c for c in normalized_token if unicodedata.category(c) != 'Mn'])
            tokens_sin_acentos.append(token_sin_acento)

    # 4. Eliminar tokens de 1 carácter
    tokens_filtrados = [token for token in tokens_sin_acentos if len(token) > 1]

    if generar_bigramas:
        # 5. Generar bigramas
        bigramas = [" ".join(bigrama) for bigrama in zip(tokens_filtrados, tokens_filtrados[1:])]
        return bigramas
    else:
        # 6. Devolver solo unigramas
        return tokens_filtrados


In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def generar_bigramas(texto):
    """Genera bigramas a partir de un texto."""

    # Generación de bigramas
    bigramas = list(nltk.bigrams(texto))
    return bigramas


In [4]:
df_preprocesado = pd.read_csv('df_preprocesado.csv', sep=';')

In [30]:
df_preprocesado['texto_preprocesado'] = df_preprocesado['texto'].apply(preprocesar_texto)

In [8]:
df_preprocesado['bigramas_preprocesado'] = df_preprocesado['texto'].apply(lambda x: preprocesar_textos(x, generar_bigramas=True))

In [18]:
df_preprocesado['bi_nltk_preprocesado'] = df_preprocesado['texto_preprocesado'].apply(ast.literal_eval).apply(generar_bigramas)

In [31]:
df_preprocesado['texto_preprocesado']

0        [equipo, dirigir, aprovechar, servicio, dejar,...
1        [instalacion, deportivo, gimnasia, deportivo, ...
2        [dudar, permiso, hola, escribir, nombre, grupo...
3        [noche, jueves, viernes, sabado, imposible, do...
4        [falta, instalacion, deportivo, familia, amant...
                               ...                        
66452    [reposicion, reparacion, cubo, basura, agradec...
66453    [baldosa, defectuoso, justo, puerta, vivienda,...
66454    [aviso, reclamacion, respuesta, poner, queja, ...
66455    [recogida, basura, mostrar, malestar, contened...
66456    [residencia, xiort, madre, rafols, dirigir, de...
Name: texto_preprocesado, Length: 66455, dtype: object

In [9]:
df_preprocesado['bigramas_preprocesado']

0        [equipo dirijo, dirijo aprovechar, aprovechar ...
1        [instalacion deportivo, deportivo gimnasia, gi...
2        [dudar permiso, permiso hola, hola escribir, e...
3        [noche jueves, jueves viernes, viernes sabado,...
4        [falta instalacion, instalacion deportivo, dep...
                               ...                        
66450    [reposicion reparacion, reparacion cubo, cubo ...
66451    [baldosa defectuoso, defectuoso justo, justo p...
66452    [aviso reclamacion, reclamacion respuesta, res...
66453    [recogida basura, basura muestro, muestro male...
66454    [residencia xiort, xiort madre, madre rafols, ...
Name: bigramas_preprocesado, Length: 66455, dtype: object

In [19]:
df_preprocesado['bi_nltk_preprocesado']

0        [(equipo, dirigir), (dirigir, aprovechar), (ap...
1        [(instalacion, deportivo), (deportivo, gimnasi...
2        [(dudar, permiso), (permiso, hola), (hola, esc...
3        [(noche, jueves), (jueves, viernes), (viernes,...
4        [(falta, instalacion), (instalacion, deportivo...
                               ...                        
66450    [(reposicion, reparacion), (reparacion, cubo),...
66451    [(baldosa, defectuoso), (defectuoso, justo), (...
66452    [(aviso, reclamacion), (reclamacion, respuesta...
66453    [(recogida, basura), (basura, mostrar), (mostr...
66454    [(residencia, xiort), (xiort, madre), (madre, ...
Name: bi_nltk_preprocesado, Length: 66455, dtype: object

In [10]:
 df_preprocesado.to_csv('df_preprocesado.csv', sep=';', index=False)

In [32]:
dictionary_spacy = corpora.Dictionary(df_preprocesado['texto_preprocesado'])
dictionary_spacy.save('dictionary_spacy.gensim')

In [11]:
dictionary_bi_spacy = corpora.Dictionary(df_preprocesado['bigramas_preprocesado'])
dictionary_bi_spacy.save('dictionary_bi_spacy.gensim')

In [33]:
len(dictionary_spacy)

50127

In [12]:
len(dictionary_bi_spacy)

990569

In [21]:
dictionary_bi_nltk = corpora.Dictionary(df_preprocesado['bi_nltk_preprocesado'].apply(lambda lista_tuplas: [" ".join(tupla) for tupla in lista_tuplas]))

In [22]:
len(dictionary_bi_nltk)

962884

In [23]:
dictionary_bi_nltk.save('dictionary_bi_nltk.gensim')

In [34]:
# Crear una copia del diccionario para no modificar el original.
dictionary_filtrado = copy.deepcopy(dictionary_spacy)

# Aplicar filter_extremes
dictionary_filtrado.filter_extremes(no_below=7, no_above=0.7)

In [35]:
dictionary_filtrado.save('dictionary_filtrado_spacy.gensim')

In [3]:
from gensim.corpora import Dictionary, MmCorpus
dictionary = Dictionary.load('dictionary_filtrado_spacy.gensim')

In [4]:
len(dictionary)

11743

In [24]:
# Crear una copia del diccionario para no modificar el original.
dictionary_filtrado_bi = copy.deepcopy(dictionary_bi_spacy)

# Aplicar filter_extremes
dictionary_filtrado_bi.filter_extremes(no_below=7, no_above=0.7)

In [25]:
len(dictionary_filtrado_bi)

39693

In [32]:
dictionary_filtrado_bi.save('dictionary_filtrado_bi.gensim')

In [26]:
# Crear una copia del diccionario para no modificar el original.
dictionary_filtrado_bi_nltk = copy.deepcopy(dictionary_bi_nltk)

# Aplicar filter_extremes
dictionary_filtrado_bi_nltk.filter_extremes(no_below=7, no_above=0.7)

In [27]:
len(dictionary_filtrado_bi_nltk)

40542

## Tablas de frecuencia de palabras

In [36]:
# Obtener el mapeo id -> palabra
token2word = {id_: word for word, id_ in dictionary_filtrado.token2id.items()}

# Obtener la frecuencia de cada palabra
token_frequencies = dictionary_filtrado.dfs  # cfs (collection frequency statistics)

# Convertirlo en un DataFrame ordenado
df_documentos = pd.DataFrame(list(token_frequencies.items()), columns=["token_id", "frequency"])
df_documentos["word"] = df_documentos["token_id"].map(token2word)  # Mapear id a palabra

# Ordenar de mayor a menor frecuencia
df_documentos = df_documentos.sort_values(by="frequency", ascending=False).reset_index(drop=True)

print(df_documentos)

       token_id  frequency         word
0           158      28020        calle
1            33      13952       gracia
2           391      13121         zona
3            63      12784       saludo
4           320      11653    solicitar
...         ...        ...          ...
11738      7912          7     suroeste
11739      8602          7      obligen
11740      8616          7      clienta
11741      4673          7   inhumacion
11742      9507          7  escuchar el

[11743 rows x 3 columns]


In [28]:
# Obtener el mapeo id -> palabra
token2word = {id_: word for word, id_ in dictionary_filtrado_bi.token2id.items()}

# Obtener la frecuencia de cada palabra
token_frequencies = dictionary_filtrado_bi.dfs  # cfs (collection frequency statistics)

# Convertirlo en un DataFrame ordenado
df_documentos = pd.DataFrame(list(token_frequencies.items()), columns=["token_id", "frequency"])
df_documentos["word"] = df_documentos["token_id"].map(token2word)  # Mapear id a palabra

# Ordenar de mayor a menor frecuencia
df_documentos = df_documentos.sort_values(by="frequency", ascending=False).reset_index(drop=True)

print(df_documentos)

       token_id  frequency                  word
0           244       4368   solicito reparacion
1           299       3666             punto luz
2          6364       3048       matricula punto
3           241       1968      reparacion bache
4           539       1815   reparacion semaforo
...         ...        ...                   ...
39688     10861          7         vehiculo foto
39689     30652          7  estar funcionamiento
39690     30648          7         cosa habitual
39691     10847          7            caer plaza
39692     19845          7           portal dato

[39693 rows x 3 columns]


In [29]:
df_documentos[:10]

,token_id,frequency,word
0,244,4368,solicito reparacion
1,299,3666,punto luz
2,6364,3048,matricula punto
3,241,1968,reparacion bache
4,539,1815,reparacion semaforo
5,535,1789,fundido solicito
6,540,1763,semaforo fundido
7,352,1727,gracia saludo
8,2841,1674,adjunto foto
9,1110,1592,policia local


In [39]:
corpus = [dictionary_filtrado.doc2bow(text) for text in df_preprocesado['texto_preprocesado']]

In [30]:
corpus_bi = [dictionary_filtrado_bi.doc2bow(text) for text in df_preprocesado['bigramas_preprocesado']]

In [31]:
# Guardar el corpus BoW en formato .mm
MmCorpus.serialize('corpus_bi.mm', corpus_bi)

# Analisis de Servicios 

In [44]:
pares_unicos = set(zip(df_preprocesado['service_name'], df_preprocesado['service_code']))

# Mostrar los pares
for par in sorted(pares_unicos, key=lambda x: x[0]):
    print(par)

('ABONADO ARBUSTOS', 1805)
('ABONADO CESPED', 1803)
('ABONADO FLORES', 1804)
('ALCORQUES', 1402)
('ALUMBRADO DETERIORADO(ROTO)', 1106)
('APEO', 1302)
('ASPERSOR ROTO', 1501)
('AVERIA SISTEMA DE RIEGO', 1506)
('Accesibilidad web', 24182784)
('Aceite', 501)
('Acera', 10)
('Admin. Electrónica: IyD', 109740033)
('Admin. Electrónica: Tramita', 109740032)
('Admin. Electrónica: telefónica', 109740034)
('Administración Electrónica:Web', 82182145)
('Adoquines', 50)
('Agradecimientos', 1000021)
('Agua', 4030465)
('Alumbrado Publico', 5144577)
('Anclajes', 502)
('Animales domésticos muertos en domicilios', 515)
('Animales muertos en viario público', 235)
('Arbolado', 170)
('Archivo Municipal', 6029312)
('BANCOS', 1201)
('BORDILLOS', 1601)
('Bache', 61)
('Bache', 11)
('Baldosas', 20)
('Barandillas', 40)
('Basura abandonada en la vía pública', 234)
('Basura y enseres alrededor de los contenedores', 503)
('Biblioteca-hemeroteca', 6520832)
('Bibliotecas Municipales', 5144585)
('Bicicletas', 9043969)


In [45]:
df_pares = pd.DataFrame(pares_unicos, columns=['columna_1', 'columna_2'])

# Guardar el DataFrame como archivo CSV
df_pares.to_csv('pares_unicos.csv', index=False)

In [48]:
len(df_preprocesado['service_code'].unique())

232

In [19]:
df_preprocesado = pd.read_csv('df_preprocesado.csv', sep=';')

# Servicios unificados

In [2]:
import pandas as pd
df_service_code = pd.read_csv('servicios_unicos.csv', sep=",",header=None)

In [3]:
df_service_code.head(20)

,0,1,2
0,INTERNAL,2,2
1,Acera,10,4849669
2,Bache,11,4849669
3,Deteriorada,12,4849669
4,Elevada,13,4849669
5,Hundida,14,4849669
6,Baldosas,20,4849669
7,Faltan,21,4849669
8,Rotas,22,4849669
9,Bordillos,30,4849669


In [15]:
df_service_code = df_service_code.rename(columns={2: 'service_code_unificado'})

In [12]:
df_service_code[1]

0              2
1             10
2             11
3             12
4             13
         ...    
227    109740033
228    109740034
229    118587392
230    184647680
231    188088321
Name: 1, Length: 232, dtype: int64

In [20]:
import pandas as pd

def agregar_code_service_unificado(df_preprocesado, df_service_code):
    # Crear un diccionario para mapear service_code a service_code_unificado
    mapeo_codigos = dict(zip(df_service_code['service_code'], df_service_code['service_code_unificado']))

    # Agregar la nueva columna utilizando el mapeo
    df_preprocesado['code_service_unificado'] = df_preprocesado['service_code'].map(mapeo_codigos)

    return df_preprocesado

# Ejemplo de uso:
# Supongamos que tienes tus DataFrames df_preprocesado y df_service_code cargados
# df_preprocesado = pd.read_csv("tu_archivo_preprocesado.csv")
# df_service_code = pd.read_csv("tu_archivo_service_code.csv")

df_preprocesado_con_codigo = agregar_code_service_unificado(df_preprocesado, df_service_code)
# print(df_preprocesado_con_codigo)

In [33]:
len(df_preprocesado['code_service_unificado'].unique())

61